In [1]:
import xml.etree.ElementTree as ET
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np


import string
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
import networkx as nx


import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
def xml_to_df(xmlfilename):
    tree = ET.parse(xmlfilename, parser=ET.XMLParser(encoding='utf-8'))
    root = tree.getroot()

    def get_attr(root, attributes):
        for child in root.iter('row'):
            attributes.append(child.attrib)
        return attributes

    attributes = get_attr(root,[])
    print(attributes)
    df = pd.DataFrame(attributes)
    return df

In [3]:
df = xml_to_df("dataset/Posts.xml").to_csv("post_question.csv")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
def answerer_questioner_t(postquescsv, UserXmlfilename):
    # User u answer User v's question at time t
    df = pd.read_csv(postquescsv)
    df.Body = df.Body.astype('str')
    df1 = df[df['PostTypeId'] == 1]
    total_ques = len(df1)
    print(f"Total questions {total_ques}")
    df1 = df1[['Id','Body','CreationDate','OwnerUserId']]
    
    df1.rename(columns = {'Id':'parentid','CreationDate':'qcreationdate','Body':'qbody'}, inplace = True)
    df1.reset_index()
    df2 = df[df['PostTypeId'] == 2]
    df2 = df2[["Id","ParentId","CreationDate","Body"]]
    df2.rename(columns = {'ParentId':'parentid','CreationDate':'acreationdate','Body':'abody'}, inplace = True)
    df2.reset_index()

    df = pd.merge(df1, df2, on="parentid")
    # sort values by answer creation date
    df=df.sort_values(['acreationdate'])
    df=df.drop_duplicates(subset=['parentid'])
    df['qcreationdate'] = pd.to_datetime(df['qcreationdate'], format='mixed')
    df['acreationdate'] = pd.to_datetime(df['acreationdate'], format='mixed')
    df["response_time"] = (df.acreationdate - df.qcreationdate) / pd.Timedelta(minutes=1)
    
    df = df[df['response_time']>=0]
    
    df = df[["parentid","OwnerUserId","Id","response_time"]]
    df = df.dropna()
    user = xml_to_df(UserXmlfilename)
    # changing the datatype of user dataframe
    user = user.astype({"Id":'int64',"DisplayName":str,"Reputation":'int64'})
    user =  user[["Id","DisplayName","Reputation"]]
    user = user.dropna()    
    user.rename(columns = {'Id':'OwnerUserId'}, inplace = True)
    
    df = pd.merge(df,user, on="OwnerUserId")
    df.rename(columns = {'OwnerUserId':'questioner'}, inplace = True)
    #df.rename(columns = {'DisplayName':'questioner'}, inplace = True)
    
    df2 = pd.read_csv(postquescsv)
    answer_all = df2[df2['PostTypeId'] == 2]
    answer_all = answer_all[["Id","OwnerUserId"]]
    answer_all = answer_all.reset_index()
    
    df3 = pd.merge(df,answer_all,on="Id")
    df4 = df3
   # df4 = pd.merge(df3,user, on="OwnerUserId")
    df4.rename(columns = {'OwnerUserId':'answerer'}, inplace = True)
    print(df4.columns)
    df4 = df4[['parentid','questioner','Id','answerer','response_time']]
    df4.reset_index(inplace = True)

    return df4

In [5]:
df = answerer_questioner_t("post_question.csv","dataset/Users.xml")

Total questions 51772


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Index(['parentid', 'questioner', 'Id', 'response_time', 'DisplayName',
       'Reputation', 'index', 'answerer'],
      dtype='object')


In [10]:
main_df = df[["questioner",'answerer','response_time']]

In [11]:
main_df.head()

,questioner,answerer,response_time
0,14.0,13.0,300.546883
1,14.0,24.0,642.220283
2,14.0,24.0,847.843950
3,14.0,24.0,1265.539500
4,14.0,94.0,174.088117


In [12]:
len(main_df)

34788

In [13]:
main_df['response_time'] = pd.to_numeric(main_df['response_time'], errors='coerce')  # Handle non-numeric values


In [14]:
len(main_df)

34788

In [15]:
weight_for_zero = 0.1
main_df['weight'] = main_df['response_time'].apply(lambda x:1/x if x > 0 else weight_for_zero)


In [18]:
max(main_df['weight'])

3791810359.7936506

In [20]:
main_df.columns

Index(['questioner', 'answerer', 'response_time', 'normalized_response_time',
       'weight'],
      dtype='object')

In [21]:
main_df = main_df[['questioner', 'answerer','weight']]

In [22]:
G = nx.DiGraph()  # Create a directed graph

# Add nodes (questioners and answerers)
for index, row in main_df.iterrows():
    questioner = row['questioner']
    answerer = row['answerer']
    G.add_node(questioner)
    G.add_node(answerer)

# Add edges with weights
for index, row in main_df.iterrows():
    questioner = row['questioner']
    answerer = row['answerer']
    weight = row['weight']
    G.add_edge(questioner, answerer, weight=weight)

In [23]:
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 24677
Number of edges: 31122


In [24]:
degree_centrality_scores = nx.degree_centrality(G)


In [25]:
degree_centrality_scores

{14.0: 0.0016615334738207166,
 13.0: 8.105041335710812e-05,
 24.0: 0.00012157562003566218,
 94.0: 0.00012157562003566218,
 91.0: 0.005227751661533474,
 19.0: 0.00129680661371373,
 89.0: 0.00020262603339277031,
 nan: 0.0,
 113.0: 0.00012157562003566218,
 115.0: 0.0004457772734640947,
 nan: 0.0,
 35.0: 0.00036472686010698656,
 129.0: 0.00020262603339277031,
 32.0: 0.0007699789268925272,
 136.0: 0.00012157562003566218,
 125.0: 0.00016210082671421625,
 102.0: 0.00012157562003566218,
 7.0: 4.052520667855406e-05,
 180.0: 4.052520667855406e-05,
 188.0: 4.052520667855406e-05,
 193.0: 0.00016210082671421625,
 212.0: 8.105041335710812e-05,
 187.0: 0.043078294699302966,
 252.0: 8.105041335710812e-05,
 192.0: 0.00024315124007132436,
 149.0: 0.011225482249959475,
 nan: 0.0,
 304.0: 8.105041335710812e-05,
 258.0: 0.0017425838871778247,
 348.0: 4.052520667855406e-05,
 nan: 0.0,
 186.0: 0.0009320797536067435,
 312.0: 0.0019452099205705948,
 402.0: 0.00020262603339277031,
 414.0: 0.007456638028853948,
